In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

# Chrome 웹드라이버 설정
chrome_options = Options()
# chrome_options.add_argument('--headless')  # 백그라운드에서 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 스타벅스 매장찾기 페이지 접속
url = "https://www.starbucks.co.kr/store/store_map.do"
driver.get(url)

# 수집한 매장 정보를 저장할 리스트
store_list = []

try:
    # 매장 번호 1~20까지 순회
    for store_id in range(1, 21):
        try:
            # JavaScript 함수 실행하여 매장 상세정보 팝업 열기
            script = f"getStoreDetail('{store_id}')"
            driver.execute_script(script)
            
            # 매장 정보 팝업이 로드될 때까지 최대 3초 대기
            wait = WebDriverWait(driver, 3)
            store_info = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "shopArea_pop01")))
            
            # 1. 기본 정보 추출 (매장명, 주소, 전화번호)
            store_name = store_info.find_element(By.CSS_SELECTOR, "header.titl h6").text
            address = store_info.find_element(By.CSS_SELECTOR, "dl.shopArea_info dd").text
            tel = store_info.find_elements(By.CSS_SELECTOR, "dl.shopArea_info")[1].find_element(By.TAG_NAME, "dd").text
            
            # 2. 영업시간 정보 추출
            try:
                time_info = store_info.find_element(By.CLASS_NAME, "date_time").text
            except:
                time_info = "정보 없음"
            
            # 3. 서비스 정보 추출 (매장에서 제공하는 서비스)
            try:
                # 서비스 섹션 찾기 (3번째 dl.shopArea_info)
                service_section = store_info.find_elements(By.CSS_SELECTOR, "dl.shopArea_info")[2]
                # 서비스 아이콘 이미지들의 alt 텍스트 추출
                service_images = service_section.find_elements(By.TAG_NAME, "img")
                services = [img.get_attribute('alt') for img in service_images]
            except:
                services = []
                
            # 4. 위치 및 시설 정보 추출
            try:
                # 시설 섹션 찾기 (4번째 dl.shopArea_info)
                facility_section = store_info.find_elements(By.CSS_SELECTOR, "dl.shopArea_info")[3]
                # 시설 아이콘 이미지들의 alt 텍스트 추출
                facility_images = facility_section.find_elements(By.TAG_NAME, "img")
                facilities = [img.get_attribute('alt') for img in facility_images]
            except:
                facilities = []

            # 5. 주차 정보 추출
            try:
                parking_info = store_info.find_elements(By.CSS_SELECTOR, "dl.shopArea_info")[4].find_element(By.TAG_NAME, "dd").text
            except:
                parking_info = "정보 없음"

            # 6. 오시는 길 정보 추출
            try:
                directions = store_info.find_elements(By.CSS_SELECTOR, "dl.shopArea_info")[5].find_element(By.TAG_NAME, "dd").text
            except:
                directions = "정보 없음"
                
            # 7. 추출한 정보를 딕셔너리로 구성
            store_data = {
                "id": store_id,
                "name": store_name,
                "address": address,
                "tel": tel,
                "business_hours": time_info,
                "services": services,           # 제공 서비스 목록
                "facilities": facilities,       # 매장 시설 정보
                "parking_info": parking_info,   # 주차 관련 정보
                "directions": directions        # 오시는 길 안내
            }
            
            # 리스트에 매장 정보 추가
            store_list.append(store_data)
            print(f"매장 {store_id} 정보 추출 완료: {store_name}")
            
            # 매장 정보 팝업 닫기 버튼 클릭
            close_button = store_info.find_element(By.CSS_SELECTOR, "p.btn_pop_close a.isStoreViewClosePop")
            close_button.click()
            
            # 팝업이 완전히 닫힐 때까지 짧게 대기
            time.sleep(0.5)
            
        except Exception as e:
            print(f"매장 {store_id} 정보 없음 또는 오류 발생")
            continue
            
    # 수집한 데이터를 JSON 파일로 저장
    with open("starbucks_stores.json", "w", encoding="utf-8") as f:
        json.dump(store_list, f, ensure_ascii=False, indent=2)
    
    print(f"총 {len(store_list)}개 매장 정보 수집 완료")

finally:
    # 브라우저 종료
    driver.quit()

매장 1 정보 없음 또는 오류 발생
매장 2 정보 없음 또는 오류 발생
매장 3 정보 추출 완료: 학여울역
매장 4 정보 없음 또는 오류 발생
매장 5 정보 없음 또는 오류 발생
매장 6 정보 추출 완료: 둔산은하수
매장 7 정보 추출 완료: 신세계강남7층
매장 8 정보 없음 또는 오류 발생
매장 9 정보 추출 완료: 명동눈스퀘어
매장 10 정보 추출 완료: 숭실대
매장 11 정보 없음 또는 오류 발생
매장 12 정보 추출 완료: 광교
매장 13 정보 추출 완료: 구인천공항중앙
매장 14 정보 추출 완료: 베네시티
매장 15 정보 없음 또는 오류 발생
매장 16 정보 없음 또는 오류 발생
매장 17 정보 추출 완료: 천호역
매장 18 정보 추출 완료: 홍대갤러리
매장 19 정보 추출 완료: 아크플레이스
매장 20 정보 추출 완료: 여주P.아울렛입구
총 12개 매장 정보 수집 완료
